In [10]:
import pandas as pd
import numpy as np
import sqlite3 as sql
import sqlalchemy
import nltk
from sqlalchemy import create_engine
nltk.download('stopwords')
from nltk.corpus import stopwords
from sklearn.preprocessing import StandardScaler
from scipy.sparse import csr_matrix

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
from lightfm.cross_validation import random_train_test_split
from lightfm.evaluation import auc_score, precision_at_k, recall_at_k
from lightfm import LightFM
from skopt import forest_minimize

https://github.com/kapadias/mediumposts/blob/master/recommender/published_notebooks/recommendation_python_lightfm.ipynb
https://realpython.com/build-recommendation-engine-collaborative-filtering/
https://github.com/benhamner/Metrics/blob/master/Python/ml_metrics/average_precision.py
https://github.com/NicolasHug/Surprise/blob/master/examples/top_n_recommendations.py

In [2]:
connection = sql.connect('songs.db')
cursor = connection.cursor()
engine = sqlalchemy.create_engine('sqlite:///songs.db') 

In [8]:
query1 = """select user_id, id, count
    from train_triplets 
"""

In [7]:
query1 = """select user_id, tt.id, count 
    from train_triplets tt
    join msd_tagtraum mt on mt.id = tt.id
"""

In [8]:
df = pd.read_sql(sqlalchemy.text(query1), engine)

In [9]:
user_id = list(df['user_id'])
user_dict = {}
counter = 0 
for i in user_id:
    user_dict[i] = counter
    counter += 1

In [12]:
user_book_interaction_csr = csr_matrix(df[['id', 'count']].values)
user_book_interaction_csr

<30168816x2 sparse matrix of type '<class 'numpy.int64'>'
	with 60337632 stored elements in Compressed Sparse Row format>

In [10]:
df1 = df.copy()

In [5]:
scaler = StandardScaler()
df[['count']] = scaler.fit_transform(df[['count']])

In [34]:
df.tail(5)

,user_id,id,count
30168811,b7815dbb206eb2831ce0fe040d0aa537e2e800f7,640312,-0.280533
30168812,b7815dbb206eb2831ce0fe040d0aa537e2e800f7,407473,-0.280533
30168813,b7815dbb206eb2831ce0fe040d0aa537e2e800f7,851477,-0.280533
30168814,b7815dbb206eb2831ce0fe040d0aa537e2e800f7,210428,-0.280533
30168815,b7815dbb206eb2831ce0fe040d0aa537e2e800f7,489324,0.036999


In [12]:
df['count'] = df['count'].astype('float16')

In [10]:
df['id'] = df['id'].astype('int32')

Человек я простой, с нормами поведения в ладах. Слежу за личной гигиеной. Скромен, легок на подъем. Воспитание и манеры присутствуют. Юмор наше все. Занимаюсь саморазвитием. Зависимостей от игр или алкоголя нет. Легок на подъем. Стремлюсь к карьерному росту. Доход средний. Увлекаюсь нумизматикой

Человек я простой, с нормами поведения в ладах. Слежу за личной гигиеной. Скромен, легок на подъем. Воспитание и манеры присутствуют. Юмор наше все. Занимаюсь саморазвитием. Стремлюсь к карьерному росту. Доход -хватает для съема квартиры возле работы. Своих машины, квартиры нет. 

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30168816 entries, 0 to 30168815
Data columns (total 3 columns):
 #   Column   Dtype  
---  ------   -----  
 0   user_id  object 
 1   id       int32  
 2   count    float16
dtypes: float16(1), int32(1), object(1)
memory usage: 402.8+ MB


In [14]:
df4 = df.unstack(1)

In [32]:

user_book_interaction = pd.pivot_table(df, index='user_id', columns='id', values='count', aggfunc=np.sum, fill_value=0)

# fill missing values with 0
# user_book_interaction = user_book_interaction.fillna(0)

In [33]:
user_book_interaction

id,4823,16977,17952,23516,25271,26945,28086,28406,28758,30100,...,971392,973705,973871,975473,979394,980487,980969,984709,990086,996238
user_id,,,,,,,,,,,,,,,,,,,,,
17aa9f6dbdf753831da8f38c71b66b64373de613,0,1,0,0,0,2,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4bd88bfb25263a75bbdd467e74018f4ae570e5df,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
85c1f87fea955d09b4bec2e36aee110927aedf9a,0,0,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
8937134734f869debcab8f23d77465b4caaa85df,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
969cc6fb74e076a68e36a04409cb9d3765757508,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
9bb911319fbc04f01755814cb5edb21df3d1a336,0,0,0,0,0,0,0,0,0,0,...,0,0,0,2,0,0,0,0,0,0
9d6f0ead607ac2a6c2460e4d14fb439a146b7dec,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,2,0,0,0
b64cdd1a0bd907e5e00b39e345194768e330d652,0,0,0,1,2,0,0,1,0,3,...,0,0,2,0,0,0,0,0,0,0
b80344d063b5ccb3212f76538f3d9e43d87dca9e,2,0,0,0,0,0,0,0,1,0,...,1,1,0,0,0,0,0,0,5,0


In [14]:
df.head(5)

,user_id,id,count
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,397230,-0.290060
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,393160,-0.290060
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,4823,-0.134279
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,973705,-0.290060
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,428124,-0.290060


In [27]:
query2 = """select max(user_id), genre1, sum(count)
from
    (select user_id, tt.id, mt.genre1, count 
    from train_triplets tt
    join msd_tagtraum mt on mt.id = tt.id
    limit 250
    )aa
group by genre1
order by user_id
"""

In [28]:
df2 = pd.read_sql(sqlalchemy.text(query2), engine)

In [29]:
df2

,max(user_id),genre1,sum(count)
0,b64cdd1a0bd907e5e00b39e345194768e330d652,Jazz,7
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,Blues,11
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,Folk,1
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,Latin,8
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,Rap,100
5,b80344d063b5ccb3212f76538f3d9e43d87dca9e,RnB,5
6,b80344d063b5ccb3212f76538f3d9e43d87dca9e,World,8
7,e006b1a48f466bf59feefed32bec6494495a4436,Country,2
8,e006b1a48f466bf59feefed32bec6494495a4436,Electronic,112
9,e006b1a48f466bf59feefed32bec6494495a4436,Metal,3


In [7]:

pd.read_sql(sqlalchemy.text('SELECT * FROM train_triplets limit 3'), engine)

,id,user_id,song_id,count
0,397230,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAKIMP12A8C130995,1
1,393160,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAPDEY12A81C210A9,1
2,4823,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBBMDR12A8C13253B,2


In [4]:
cursor.execute('''CREATE TABLE IF NOT EXISTS unique_tracks
              (id INTEGER PRIMARY KEY AUTOINCREMENT, track_id TEXT, song_id TEXT, artist text, song_name text)''')

In [5]:
def read_file(filename):
    data = []
    with open(filename, 'r',  encoding='utf-8') as f:
        for i, line in enumerate(f):
            l = line.split('<SEP>')
            # print(l)
            res = {'track_id': l[0] , 'song_id': l[1] , 'artist': l[2], 'song_name': l[3].strip('\n')}
            # print(res)
            # break
            data.append(res)
    pd.DataFrame(data).to_sql('unique_tracks', con=engine,if_exists='append', index=False)

In [6]:
read_file('p02_unique_tracks.txt')

In [52]:
cursor.execute('''CREATE TABLE IF NOT EXISTS train_triplets
              (id INTEGER, user_id TEXT, song_id TEXT, count INT)''')

In [16]:
cursor.execute('''drop table  msd_tagtraum
              ''')

In [62]:
def read_file(filename, iop):
    data = []
    df1 = pd.read_sql(sqlalchemy.text('''SELECT distinct id, song_id FROM unique_tracks'''), engine)
    # print(df1)
    with open(filename, 'r') as f:
        for i, line in enumerate(f, start=iop):
            l = line.split('\t')
            res = {'user_id': l[0] , 'song_id': l[1] , 'count': int(l[2].strip('\n'))}
            data.append(res)
            if i % 100000 == 0:
                iop = i + 1
                print(iop)
                df = pd.DataFrame(data)
                # lst = df['song_id'].to_list()
                df2 = df.merge(right=df1, on='song_id', how='left')
                df2.to_sql('train_triplets', con=engine,if_exists='append', index=False)
                # print(df2)
                # break
                # print(df)
                # for index, row in df.iterrows():
                #     # print(type(df1.iloc[0][0]))
                #     df['id'][index] = df1.iloc[0][0]
                #     # print(df)
                # df.to_sql('train_triplets', con=engine,if_exists='append', index=False)
                data = []
    # return 1
    df = pd.DataFrame(data)
                # lst = df['song_id'].to_list()
    df2 = df.merge(right=df1, on='song_id', how='left')
    df2.to_sql('train_triplets', con=engine,if_exists='append', index=False)

In [63]:
df = read_file('train_triplets.txt', 0)

1
100001
200001
300001
400001
500001
600001
700001
800001
900001
1000001
1100001
1200001
1300001
1400001
1500001
1600001
1700001
1800001
1900001
2000001
2100001
2200001
2300001
2400001
2500001
2600001
2700001
2800001
2900001
3000001
3100001
3200001
3300001
3400001
3500001
3600001
3700001
3800001
3900001
4000001
4100001
4200001
4300001
4400001
4500001
4600001
4700001
4800001
4900001
5000001
5100001
5200001
5300001
5400001
5500001
5600001
5700001
5800001
5900001
6000001
6100001
6200001
6300001
6400001
6500001
6600001
6700001
6800001
6900001
7000001
7100001
7200001
7300001
7400001
7500001
7600001
7700001
7800001
7900001
8000001
8100001
8200001
8300001
8400001
8500001
8600001
8700001
8800001
8900001
9000001
9100001
9200001
9300001
9400001
9500001
9600001
9700001
9800001
9900001
10000001
10100001
10200001
10300001
10400001
10500001
10600001
10700001
10800001
10900001
11000001
11100001
11200001
11300001
11400001
11500001
11600001
11700001
11800001
11900001
12000001
12100001
12200001
12300001

In [20]:
pd.read_sql(sqlalchemy.text('SELECT * FROM unique_tracks limit 3'), engine)

,id,track_id,song_id,artist,song_name
0,1,TRMMMYQ128F932D901,SOQMMHC12AB0180CB8,Faster Pussy cat,Silent Night
1,2,TRMMMKD128F425225D,SOVFVAK12A8C1350D9,Karkkiautomaatti,Tanssi vaan
2,3,TRMMMRX128F93187D9,SOGTUKN12AB017F4F1,Hudson Mohawke,No One Could Ever


In [65]:
pd.read_sql(sqlalchemy.text('SELECT * FROM train_triplets limit 3'), engine)

,id,user_id,song_id,count
0,397230,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAKIMP12A8C130995,1
1,393160,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAPDEY12A81C210A9,1
2,4823,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBBMDR12A8C13253B,2


In [72]:
query1 = """select ut.artist, ut.song_name, aa.count
from
    (select id, sum(count) as count 
    from train_triplets
    group by id
    order by 2 desc
    limit 250
    )aa
left join unique_tracks ut on aa.id = ut.id
order by 3 desc, 1, 2
"""

In [73]:
df_250 = pd.read_sql(sqlalchemy.text(query1), engine)

In [74]:
df_250

,artist,song_name,count
0,Dwight Yoakam,You're The One,726885
1,Björk,Undo,648239
2,Kings Of Leon,Revelry,527893
3,Harmonia,Sehr kosmisch,425463
4,Barry Tuckwell/Academy of St Martin-in-the-Fie...,Horn Concerto No. 4 in E flat K495: II. Romanc...,389880
...,...,...,...
245,Metallica / Marianne Faithfull,The Memory Remains,37283
246,Eminem,My Name Is,37130
247,Eminem,My Name Is,37130
248,Cat Stevens,Wild World,37110


In [17]:
cursor.execute('''CREATE TABLE IF NOT EXISTS msd_tagtraum
              (id INTEGER, track_id TEXT, genre1 TEXT, genre2 TEXT)''')

In [18]:
def read_file(filename):
    data = []
    df1 = pd.read_sql(sqlalchemy.text('''SELECT distinct id, track_id FROM unique_tracks'''), engine)
    # print(df1)
    with open(filename, 'r') as f:
        for i, line in enumerate(f):
            if line[0] == '#':
                continue
            l = line.split('\t')
            if len(l) < 3:
                genre2 = None
            else:
                genre2 = l[2].strip('\n')
            res = {'track_id': l[0] , 'genre1': l[1].strip('\n') , 'genre2': genre2}
            data.append(res)
    df = pd.DataFrame(data)
    df2 = df.merge(right=df1, on='track_id', how='left')
    df2.to_sql('msd_tagtraum', con=engine,if_exists='append', index=False)

In [19]:
df = read_file('p02_msd_tagtraum_cd2.cls')

In [20]:
pd.read_sql(sqlalchemy.text('SELECT * FROM msd_tagtraum limit 3'), engine)

,id,track_id,genre1,genre2
0,440375,TRAAAAK128F9318786,Rock,None
1,440380,TRAAAAW128F429D538,Rap,None
2,440358,TRAAABD128F429CF47,Rock,RnB


In [22]:
query2 = """select ut.artist, ut.song_name, mt.genre1, aa.count
from
    (select id, sum(count) as count 
    from train_triplets
    group by id
    order by 2 desc
    )aa
left join unique_tracks ut on aa.id = ut.id
left join msd_tagtraum mt on aa.id = mt.id
order by 4 desc, 3, 1, 2
"""

In [23]:
df_genres = pd.read_sql(sqlalchemy.text(query2), engine)

In [25]:
df_genre = df_genres[df_genres['genre1'] == 'Rock']

In [26]:
df_genre[:100]

,artist,song_name,genre1,count
1,Björk,Undo,Rock,648239
2,Kings Of Leon,Revelry,Rock,527893
3,Harmonia,Sehr kosmisch,Rock,425463
6,OneRepublic,Secrets,Rock,292642
8,Tub Ring,Invalid,Rock,268353
...,...,...,...,...
355,Metric,Gold Guns Girls,Rock,28148
361,Pearl Jam,Encore Break,Rock,27579
366,Daughtry,No Surprise,Rock,27187
372,Eric Clapton,Tears In Heaven,Rock,26999


In [4]:
cursor.execute('''CREATE TABLE IF NOT EXISTS dataset
              (id INTEGER PRIMARY KEY AUTOINCREMENT, word TEXT)''')

In [17]:
cursor.execute('''drop table if exists dataset
              ''')

In [5]:
sw = stopwords.words('english')

In [18]:
def read_file(filename):
    data = []
    # print(df1)
    with open(filename, 'r', encoding='utf8') as f:
        for i, line in enumerate(f):
            if line[0] == '#':
                continue
            if line[0] == '%':
                data = line[1:].split(',')
                data[-1] = data[-1].strip('\n')
                data1 = [(j, w) for j, w in enumerate(data, start=1) if not w in sw]
                # print(data1)
                # res = {'word': data1}
                df = pd.DataFrame(data1, columns=['id', 'word'])
                print(df)
                # break
                df.to_sql('dataset', con=engine,if_exists='append', index=False)
                break

In [19]:
df = read_file('mxm_dataset_train.txt')

        id    word
0       27    love
1       29    know
2       36    like
3       38     que
4       39    time
...    ...     ...
4879  4996   santo
4880  4997      pe
4881  4998     gee
4882  4999  colleg
4883  5000     kad

[4884 rows x 2 columns]


In [28]:
pd.read_sql(sqlalchemy.text('SELECT * FROM dataset limit 3'), engine)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      3 non-null      int64 
 1   word    3 non-null      object
dtypes: int64(1), object(1)
memory usage: 176.0+ bytes


Если слова встречаются более 10 раз допустим в песне то это песня про любовь

In [29]:
def read_file(filename):
    df1 = pd.read_sql(sqlalchemy.text('SELECT * FROM dataset'), engine)
    df2 = pd.read_sql(sqlalchemy.text('''SELECT distinct id, track_id FROM unique_tracks'''), engine)
    lst = set(df1['id'].to_list())
    df1.set_index('id', inplace=True, drop=True)
    ui = []
    # print(df1)
    with open(filename, 'r', encoding='utf8') as f:
        for i, line in enumerate(f):
            if line[0] == '#':
                continue
            elif line[0] == '%':
                continue
                # col = line[1:].split(',')
                # col.insert(0, 'track_id')
                # col[-1] = col[-1].strip('\n')
                # df5 = pd.DataFrame(columns=col, dtype=np.uint8)
                # df5.set_index(['track_id'], inplace=True, drop=True)
                # print(df4)
                # print(df4.info())
                # df4.to_sql('mxmx', con=engine,if_exists='append', index=True)
            else:
                data = line.split(',')
                data[-1] = data[-1].strip('\n')
                # print(data)
                for j, dat in enumerate(data):
                    res = {}
                    # print(dat[i])
                    if j == 0:
                        continue
                    elif j == 1:
                        continue
                    else:
                        ras = dat.split(':')
                        ids = int(ras[0])
                        count = int(ras[1])
                        # df1 = pd.read_sql(sqlalchemy.text('SELECT * FROM dataset where id = :ids'), engine, params={'ids': ids})
                        # print(ids)
                        # res['id'] = ids
                        # res['count'] = count
                        if ids in lst:
                            res['track_id'] = data[0]
                            res['dd'] = df1[df1.index == ids].iloc[0][0]
                            res['count'] = count
                            ui.append(res)
                            # print(ids)
                            # print(df1[df1['id'] == ids])
                            # print(df1[df1.index == ids].iloc[0][0])
                            # res[df1[df1.index == ids].iloc[0][0]] = count
                            # print(res)
                        else:
                            continue
            if i % 40000 == 0:
                iop = i + 1
                print(iop)
                # print(res)
                df = pd.DataFrame(ui, dtype=np.uint8)
                df3 = df.merge(right=df2, on='track_id', how='left')
                df4 = df3[['id', 'dd',   'count']]
                # print(df4)
                # print(df4.info())
                df4.to_sql('mxmx', con=engine, if_exists='append', index=False)
                ui = []
                # return 9
                # df.set_index(['track_id'], inplace=True, drop=True)
                # df.dropna(axis=1, how='all')
                # # df = df.convert_dtypes()
                # df4 = pd.concat([df5, df])
                # df4.fillna(0, inplace=True)
                # df4 = df4.astype('uint8')
                # # df4.to_excel(f'{iop}')
                # df4 = pd.DataFrame()
                # print('RR')
                # # print(df4)
                # # print(df4.info())
                # ui = []
                # # return 9
        df = pd.DataFrame(ui, dtype=np.uint8)
        df3 = df.merge(right=df2, on='track_id', how='left')
        df4 = df3[['id', 'dd',   'count']]
        df4.to_sql('mxmx', con=engine, if_exists='append', index=False)

In [3]:
pd.read_sql(sqlalchemy.text('SELECT * FROM mxmx limit 3'), engine)

,id,dd,count
0,440379,like,2
1,440379,de,1
2,440379,got,1


In [69]:
cursor.execute('''drop table if exists mxmx
              ''')

In [30]:
df7 = read_file('mxm_dataset_train.txt')

40001


C:\Users\user\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:1: FutureWarning: Could not cast to uint8, falling back to object. This behavior is deprecated. In a future version, when a dtype is passed to 'DataFrame', either all columns will be cast to that dtype, or a TypeError will be raised
  """Entry point for launching an IPython kernel.
C:\Users\user\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:57: FutureWarning: Values are too large to be losslessly cast to uint8. In a future version this will raise OverflowError. To retain the old behavior, use pd.Series(values).astype(uint8)


80001
120001
160001
200001


C:\Users\user\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:78: FutureWarning: Values are too large to be losslessly cast to uint8. In a future version this will raise OverflowError. To retain the old behavior, use pd.Series(values).astype(uint8)


In [33]:
query3 = """select dd as word, sum(count) as count 
    from mxmx
    group by dd
"""

In [34]:
df1 = pd.read_sql(sqlalchemy.text(query3), engine)

In [40]:
df2 = df1[25:]

In [43]:
df3 = df2[:-17]

In [49]:
dict_word = {}
for index, row in df3.iterrows():
    dict_word[row['word']] = row['count']

In [50]:
dict_word

{'aaah': 554,
 'aah': 1428,
 'aan': 714,
 'ab': 963,
 'abandon': 1303,
 'aber': 1696,
 'abl': 1560,
 'aboard': 514,
 'abov': 10495,
 'abr': 475,
 'absenc': 416,
 'absolut': 1015,
 'absurd': 458,
 'abus': 1664,
 'abyss': 551,
 'acaba': 536,
 'acabar': 416,
 'accept': 2069,
 'accid': 596,
 'accus': 619,
 'ace': 553,
 'ach': 2373,
 'achiev': 752,
 'acid': 640,
 'across': 7404,
 'act': 7272,
 'action': 3051,
 'actor': 454,
 'actual': 668,
 'ad': 1764,
 'adam': 603,
 'add': 1227,
 'addict': 2010,
 'adentro': 375,
 'adesso': 855,
 'adio': 713,
 'adió': 681,
 'admir': 517,
 'admit': 1954,
 'ador': 2082,
 'advanc': 551,
 'adventur': 596,
 'advic': 1375,
 'afar': 402,
 'affair': 1463,
 'affect': 1473,
 'afford': 1124,
 'afraid': 8969,
 'africa': 780,
 'afternoon': 1592,
 'age': 4543,
 'ago': 4440,
 'agoni': 988,
 'agora': 791,
 'agre': 1510,
 'agua': 1110,
 'ah': 18174,
 'aha': 1036,
 'ahead': 4410,
 'ahh': 2757,
 'ahhh': 817,
 'ahi': 744,
 'ahora': 4051,
 'ahí': 648,
 'ai': 23188,
 'aid': 611,

In [51]:
from gensim.models import Word2Vec

w2v_model = Word2Vec(
    min_count=200,
    vector_size = 200,
    window=4,
    alpha=0.05,
    min_alpha=0.0007,
    sample=6e-5,
    seed=21,
    sg=1)

In [52]:
w2v_model.build_vocab_from_freq(dict_word)

In [53]:
w2v_model.train(dict_word, total_examples=w2v_model.corpus_count, epochs=20)

Exception in thread Thread-6:
Traceback (most recent call last):
  File "c:\Users\user\AppData\Local\Programs\Python\Python37\lib\threading.py", line 926, in _bootstrap_inner
    self.run()
  File "c:\Users\user\AppData\Local\Programs\Python\Python37\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "c:\Users\user\AppData\Local\Programs\Python\Python37\lib\site-packages\gensim\models\word2vec.py", line 1219, in _job_producer
    epoch_progress = 1.0 * pushed_words / total_words
TypeError: unsupported operand type(s) for /: 'float' and 'NoneType'



In [12]:
sum_df = df1.sum(axis=0)

In [14]:
dict_word = dict(sum_df)

In [18]:
w2v_model.train(dict_word, total_examples=w2v_model.corpus_count, epochs=20)

Exception in thread Thread-6:
Traceback (most recent call last):
  File "c:\Users\user\AppData\Local\Programs\Python\Python37\lib\threading.py", line 926, in _bootstrap_inner
    self.run()
  File "c:\Users\user\AppData\Local\Programs\Python\Python37\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "c:\Users\user\AppData\Local\Programs\Python\Python37\lib\site-packages\gensim\models\word2vec.py", line 1219, in _job_producer
    epoch_progress = 1.0 * pushed_words / total_words
TypeError: unsupported operand type(s) for /: 'float' and 'NoneType'



: 

In [120]:
pd.read_sql(sqlalchemy.text('SELECT count(*) FROM unique_tracks'), engine)

,count(*)
0,1000000


In [123]:
df1.to_excel('mxm.xlsx', sheet_name='dd')

In [119]:
df1

,i,the,you,to,and,a,me,it,not,in,...,writer,motivo,bake,insist,wel,santo,pe,gee,colleg,kad
track_id,,,,,,,,,,,,,,,,,,,,,
TRAAAAV128F421A322,6,4,2,2,5,3,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
TRAAABD128F429CF47,10,0,17,8,2,2,1,3,2,3,...,0,0,0,0,0,0,0,0,0,0
TRAAAED128E0783FAB,28,15,2,12,22,2,2,4,2,1,...,0,0,0,0,0,0,0,0,0,0
TRAAAEF128F4273421,5,4,3,2,1,11,0,0,4,0,...,0,0,0,0,0,0,0,0,0,0
TRAAAEW128F42930C0,4,0,0,5,7,2,4,0,1,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TRZZZWS128F429CF87,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
TRZZZXA128F428ED56,1,13,6,5,4,6,0,0,4,4,...,0,0,0,0,0,0,0,0,0,0
TRZZZXV128F4289747,13,3,17,5,9,0,8,2,1,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
df1.to_sql('mxmx', con=engine,if_exists='append', index=False)

In [14]:
df1

,track_id,i,the,you,to,and,a,me,it,not,...,writer,motivo,bake,insist,wel,santo,pe,gee,colleg,kad
0,TRAAAAV128F421A322,6,4,2,2,5,3,1,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,TRAAABD128F429CF47,10,NaN,17,8,2,2,1,3,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,TRAAAED128E0783FAB,28,15,2,12,22,2,2,4,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,TRAAAEF128F4273421,5,4,3,2,1,11,NaN,NaN,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [56]:
data = pd.DataFrame([{'aer': ['но', 'каждый', 'хотеть', 'исправлять']}, {'aer': ['скучать', 'вправлять', 'мозги', 'равно', 'скучать']},{'aer':['вот', 'школа', 'говно', 'это', 'идти']}])

In [5]:
data = pd.DataFrame([{'dd': {'но':1, 'каждый':2, 'хотеть': 2, 'исправлять':1}}, {'dd': {'скучать': 1, 'вправлять': 2, 'мозги':2, 'равно':1, 'скучать': 2}},{'dd':{'вот':3, 'школа':8, 'говно':8, 'это':1, 'идти':1}}])

In [7]:
d = {'col1': [0, 1, 2, 3], 'col2': pd.Series([2, 3], index=[2, 3])}
ff = pd.DataFrame(data=d, index=[0, 1, 2, 3])

In [8]:
ff.sum()

col1    6.0
col2    5.0
dtype: float64

In [9]:
ff

,col1,col2
0,0,NaN
1,1,NaN
2,2,2.0
3,3,3.0


In [84]:
data = [{'но':1, 'каждый':2, 'хотеть': 2, 'исправлять':1},{'скучать': 1, 'вправлять': 2, 'мозги':2, 'равно':1, 'скучать': 2},{'вот':3, 'школа':8, 'говно':8, 'это':1, 'идти':1}]

In [102]:
data = {'но':1, 'каждый':2, 'хотеть': 2, 'исправлять':1,'скучать': 1, 'вправлять': 2, 'мозги':2, 'равно':1, 'скучать': 2,'вот':3, 'школа':8, 'говно':8, 'это':1, 'идти':1}

vk1.a.b9vf7WunT9_O-dalUspCNM82kuBSOJb4iqU7FIFkuU6UWmrAzzxZQycoKGcyYCm-5wtvCbJ0VgHbp0BG45Px5bazZ4lp2BL7cB8Q7sGl_qJwXxiKTWaOisl9iR89gDD1tJL1_-Z_9HoR979Co9Ub5ComXYjcKl_ZQzkpJdC8d3tN43Mj9gVAjWTGq3-aVcsI

In [103]:
data

{'но': 1,
 'каждый': 2,
 'хотеть': 2,
 'исправлять': 1,
 'скучать': 2,
 'вправлять': 2,
 'мозги': 2,
 'равно': 1,
 'вот': 3,
 'школа': 8,
 'говно': 8,
 'это': 1,
 'идти': 1}

In [104]:
from gensim.models import Word2Vec

w2v_model = Word2Vec(
    min_count=5,
    vector_size = 200,
    window=4,
    alpha=0.05,
    min_alpha=0.0007,
    sample=6e-5,
    seed=21,
    sg=1)

In [105]:
f = w2v_model.build_vocab(data)

In [106]:
f = w2v_model.build_vocab_from_freq(data)

In [98]:
data

{'но': 1,
 'каждый': 2,
 'хотеть': 2,
 'исправлять': 1,
 'скучать': 2,
 'вправлять': 2,
 'мозги': 2,
 'равно': 1,
 'вот': 3,
 'школа': 8,
 'говно': 8,
 'это': 1,
 'идти': 1}

In [107]:
w2v_model.train(data, total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)

(0, 2100)

In [108]:
for word in w2v_model.wv.key_to_index:
    print(word)

говно
школа
вот
мозги
вправлять
скучать
хотеть
каждый
идти
это
равно
исправлять
но


In [110]:
w2v_model.wv.most_similar(positive=["каждый"])

[('исправлять', 0.06797593832015991),
 ('вот', 0.03364057466387749),
 ('вправлять', 0.009391184896230698),
 ('это', 0.008315940387547016),
 ('скучать', 0.004503006115555763),
 ('идти', -0.003644465934485197),
 ('но', -0.010839185677468777),
 ('хотеть', -0.023671666160225868),
 ('школа', -0.09575348347425461),
 ('мозги', -0.11410722136497498)]

In [12]:
df1 = df.groupby(['song_id']).agg({'count' : ['sum'], 'user_id': ['max']})

In [14]:
df1.drop([('user_id', 'max')], axis=1, inplace=True)

In [17]:
df1.head(5)

,count
,sum
song_id,
SOAARXR12A8C133D15,1
SOABRAB12A6D4F7AAF,2
SOACPBY12A8C13FEF9,1
SOACWYB12AF729E581,2
SOADGFH12A8C143D89,11


In [1]:
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, roc_curve, auc, roc_auc_score
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
import os
import numpy as np
from gensim import corpora

In [2]:

from scipy import spatial

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.dummy import DummyClassifier

In [3]:
dirname = 'data'
files = os.listdir(dirname)
temp = list(map(lambda name: os.path.join(dirname, name), files))

In [4]:
temp[0]

'data\\bank_data_test.csv'

собираем тренирскую подборку

In [5]:
train_df = pd.read_csv(temp[1], sep=',')
test_df = pd.read_csv(temp[0], sep=',')

нормализуем некоторые столбцы

In [6]:
scaler = StandardScaler()
test_df[['REST_AVG_CUR', 'TURNOVER_PAYM', 'REST_AVG_PAYM', 'DEAL_YWZ_IR_MIN', 'DEAL_YWZ_IR_MAX', 'APP_REGISTR_RGN_CODE', 'LDEAL_TENOR_MAX', 'DEAL_YQZ_IR_MAX', 'DEAL_YQZ_IR_MIN', 'LDEAL_TENOR_MIN', 'MAX_PCLOSE_DATE', 'TURNOVER_CC', 'LDEAL_AMT_MONTH', 'AGE']] = scaler.fit_transform(test_df[['REST_AVG_CUR', 'TURNOVER_PAYM', 'REST_AVG_PAYM', 'DEAL_YWZ_IR_MIN', 'DEAL_YWZ_IR_MAX', 'APP_REGISTR_RGN_CODE', 'LDEAL_TENOR_MAX', 'DEAL_YQZ_IR_MAX', 'DEAL_YQZ_IR_MIN', 'LDEAL_TENOR_MIN', 'MAX_PCLOSE_DATE', 'TURNOVER_CC', 'LDEAL_AMT_MONTH', 'AGE']])
train_df[['REST_AVG_CUR', 'TURNOVER_PAYM', 'REST_AVG_PAYM', 'DEAL_YWZ_IR_MIN', 'DEAL_YWZ_IR_MAX', 'APP_REGISTR_RGN_CODE', 'LDEAL_TENOR_MAX', 'DEAL_YQZ_IR_MAX', 'DEAL_YQZ_IR_MIN', 'LDEAL_TENOR_MIN', 'MAX_PCLOSE_DATE', 'TURNOVER_CC', 'LDEAL_AMT_MONTH', 'AGE']] = scaler.fit_transform(train_df[['REST_AVG_CUR', 'TURNOVER_PAYM', 'REST_AVG_PAYM', 'DEAL_YWZ_IR_MIN', 'DEAL_YWZ_IR_MAX', 'APP_REGISTR_RGN_CODE', 'LDEAL_TENOR_MAX', 'DEAL_YQZ_IR_MAX', 'DEAL_YQZ_IR_MIN', 'LDEAL_TENOR_MIN', 'MAX_PCLOSE_DATE', 'TURNOVER_CC', 'LDEAL_AMT_MONTH', 'AGE']])

текстовые столбцы приводим к числовому формату

In [7]:
def family(strin):
    if strin == "FRIEND" or strin == 'Друг':
        return 1
    elif strin =='MOTHER' or strin == 'Мать' or strin == 'мать':
        return 2
    elif strin == 'RELATIVE' or strin == 'Близкий ро':
        return 3
    elif strin == 'OTHER' or strin == 'Дальний ро':
        return 4
    elif strin == 'BROTHER' or strin == 'Брат':
        return 5
    elif strin == 'SISTER' or strin == 'Сестра':
        return 6
    elif strin == 'FATHER' or strin == 'Отец':
        return 7
    elif strin == 'DAUGHTER' or strin == 'Дочь':
        return 8
    elif strin == 'SON' or strin == 'Сын':
        return 9
    elif strin == 'мать':
        return 10
    elif strin == 'Жена':
        return 11
    else:
        return np.nan

In [8]:
def alfa1(strin):
    if strin == "M":
        return 1
    elif strin =='D':
        return 2
    elif strin == 'V':
        return 3
    elif strin == 'T':
        return 4
    elif strin == 'm':
        return 5
    elif strin == 'N':
        return 6
    elif strin == 'v':
        return 7
    elif strin == 'W':
        return 8
    elif strin == 'd':
        return 9
    elif strin == 't':
        return 10
    elif strin == 'C':
        return 11
    elif strin == 'w':
        return 12
    else:
        return np.nan

In [9]:
def alfa2(strin):
    if strin == "SO":
        return 1
    elif strin =='JO':
        return 2
    elif strin == 'OTHER':
        return 3
    elif strin == 'RENT':
        return 4
    elif strin == 'NPRIVAT':
        return 5
    else:
        return np.nan

In [10]:
def job(strin):
    if strin == "SPECIALIST":
        return 1
    elif strin =='MANAGER':
        return 2
    elif strin == 'TOP_MANAGER':
        return 3
    elif strin == 'SELF_EMPL':
        return 4
    else:
        return np.nan

In [11]:
def binar(strin):
    if strin == "N":
        return 0
    elif strin =='Y':
        return 1
    else:
        return np.nan

In [12]:
def alfa3(strin):
    if strin == "H":
        return 1
    elif strin =='SS':
        return 2
    elif strin == 'UH':
        return 3
    elif strin == 'S':
        return 4
    elif strin == 'HH':
        return 5
    elif strin == 'h':
        return 6
    elif strin == 's':
        return 7
    elif strin == 'A':
        return 8
    elif strin == 'i':
        return 9
    elif strin == 'US':
        return 10
    elif strin == 'I':
        return 11
    elif strin == 'HI':
        return 12
    elif strin == 'a':
        return 13
    elif strin == 'e':
        return 14
    elif strin == 'E':
        return 15
    elif strin == 'AC':
        return 16
    elif strin == 'AV':
        return 17
    else:
        return np.nan

In [13]:
def typeq(strin):
    if strin == "PRIVATE":
        return 1
    elif strin =='STATE':
        return 2
    elif strin == 'INTER':
        return 3
    elif strin == 'IP':
        return 4
    else:
        return np.nan

In [14]:
def pack(strin):
    if strin == "102":
        return 1
    elif strin =='K01':
        return 2
    elif strin == 'O01':
        return 3
    elif strin == '105':
        return 4
    elif strin == '107':
        return 5
    elif strin == '103':
        return 6
    elif strin == '104':
        return 7
    elif strin == '301':
        return 8
    elif strin == '101':
        return 9
    elif strin == '109':
        return 10
    elif strin == 'M01':
        return 11
    elif strin == '108':
        return 12
    else:
        return np.nan

In [15]:
ss = set(train_df['CLNT_JOB_POSITION'].to_list())
box = {}
i = 0
for s in ss:
    if s not in box:
        box[s] = i
        i+=1

In [16]:
def job1(strin):
    if strin in box:
        return box[strin]
    else:
         return np.nan

In [17]:
def floatinf(df):
    df["CLNT_TRUST_RELATION"] = df["CLNT_TRUST_RELATION"].apply(family)
    df["APP_MARITAL_STATUS"] = df["APP_MARITAL_STATUS"].apply(alfa1)
    df["APP_KIND_OF_PROP_HABITATION"] = df["APP_KIND_OF_PROP_HABITATION"].apply(alfa2)
    df["CLNT_JOB_POSITION_TYPE"] = df["CLNT_JOB_POSITION_TYPE"].apply(job)
    df["APP_DRIVING_LICENSE"] = df["APP_DRIVING_LICENSE"].apply(binar)
    df["APP_EDUCATION"] = df["APP_EDUCATION"].apply(alfa3)
    df["APP_TRAVEL_PASS"] = df["APP_TRAVEL_PASS"].apply(binar)
    df["APP_CAR"] = df["APP_CAR"].apply(binar)
    df["APP_POSITION_TYPE"] = df["APP_POSITION_TYPE"].apply(job)
    df["APP_EMP_TYPE"] = df["APP_EMP_TYPE"].apply(typeq)
    df["APP_COMP_TYPE"] = df["APP_COMP_TYPE"].apply(typeq)
    df["PACK"] = df["PACK"].apply(pack)
    df['CLNT_JOB_POSITION'] = df['CLNT_JOB_POSITION'].apply(job1)
    return df

In [18]:
train_df = floatinf(train_df)
test_df = floatinf(test_df)

удаляем пустые столбцы

In [19]:
train_df.drop(['PRC_ACCEPTS_A_EMAIL_LINK',
'PRC_ACCEPTS_A_POS',
'PRC_ACCEPTS_A_TK',
'PRC_ACCEPTS_A_AMOBILE',
'PRC_ACCEPTS_TK',
'PRC_ACCEPTS_A_MTP',
'CNT_ACCEPTS_TK',
'PRC_ACCEPTS_A_ATM',
'PRC_ACCEPTS_MTP',
'CNT_ACCEPTS_MTP'
], axis=1, inplace=True)

In [20]:
test_df.drop(['PRC_ACCEPTS_A_EMAIL_LINK',
'PRC_ACCEPTS_A_POS',
'PRC_ACCEPTS_A_TK',
'PRC_ACCEPTS_A_AMOBILE',
'PRC_ACCEPTS_TK',
'PRC_ACCEPTS_A_MTP',
'CNT_ACCEPTS_TK',
'PRC_ACCEPTS_A_ATM',
'PRC_ACCEPTS_MTP',
'CNT_ACCEPTS_MTP'
], axis=1, inplace=True)

In [21]:
for i in train_df.columns:
    #df[i] = df[i].astype(np.float32)
    if train_df[i].isnull().any():
        train_df[i].fillna(train_df[i].median(), inplace=True)

In [22]:
for i in test_df.columns:
    #df[i] = df[i].astype(np.float32)
    if test_df[i].isnull().any():
        test_df[i].fillna(test_df[i].median(), inplace=True)

c:\Users\user\AppData\Local\Programs\Python\Python37\lib\site-packages\numpy\lib\nanfunctions.py:1117: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


In [23]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 355190 entries, 0 to 355189
Columns: 106 entries, ID to TARGET
dtypes: float64(96), int64(10)
memory usage: 287.2 MB


In [24]:
train_df.set_index('ID', inplace=True)
test_df.set_index('ID', inplace=True)

In [25]:
test_df.head(3)

,CR_PROD_CNT_IL,AMOUNT_RUB_CLO_PRC,APP_REGISTR_RGN_CODE,TURNOVER_DYNAMIC_IL_1M,CNT_TRAN_AUT_TENDENCY1M,SUM_TRAN_AUT_TENDENCY1M,AMOUNT_RUB_SUP_PRC,SUM_TRAN_AUT_TENDENCY3M,CLNT_TRUST_RELATION,REST_DYNAMIC_FDEP_1M,...,REST_DYNAMIC_CC_3M,MED_DEBT_PRC_YWZ,LDEAL_ACT_DAYS_PCT_TR3,LDEAL_ACT_DAYS_PCT_AAVG,LDEAL_DELINQ_PER_MAXYWZ,TURNOVER_DYNAMIC_CC_3M,LDEAL_ACT_DAYS_PCT_TR,LDEAL_ACT_DAYS_PCT_TR4,LDEAL_ACT_DAYS_PCT_CURR,TARGET
ID,,,,,,,,,,,,,,,,,,,,,
400980,0,0.000000,0.128378,0.0,0.3,0.288088,0.000000,0.724026,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
525062,0,0.059381,0.128378,0.0,0.3,0.288088,0.000333,0.724026,2.0,0.0,...,0.0,1.0,0.5,0.5,0.0,0.0,0.5,0.5,0.5,NaN
280316,0,0.070430,0.128378,0.0,0.3,0.288088,0.077876,0.724026,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN


In [26]:
scaler = StandardScaler()
test_df[['CLNT_JOB_POSITION', 'CLNT_SETUP_TENOR', 'CLNT_SALARY_VALUE']] = scaler.fit_transform(test_df[['CLNT_JOB_POSITION', 'CLNT_SETUP_TENOR', 'CLNT_SALARY_VALUE']])
train_df[['CLNT_JOB_POSITION', 'CLNT_SETUP_TENOR', 'CLNT_SALARY_VALUE']] = scaler.fit_transform(train_df[['CLNT_JOB_POSITION', 'CLNT_SETUP_TENOR', 'CLNT_SALARY_VALUE']])

In [49]:
train_df.describe().to_excel('fff1.xlsx')

In [23]:
train_df.head(50).to_excel('fff.xlsx')

In [27]:
tarin_df1 = train_df

создаем список характеристик

In [28]:
target = 'TARGET'
features = [x for x in list(tarin_df1.columns) if x != target]
features[0]

'CR_PROD_CNT_IL'

In [52]:
# Мы использовали последовательную модель, которая представляет собой тип модели, в которой слои укладываются линейно. Первый слой - это слой ввода, где input_dim - это количество функций, а количество выходных единиц равно 16. В скрытом слое мы используем 8 единиц вывода. Наконец, на выходном уровне у нас есть только одна единица вывода, которая возвращает вероятность оттока клиентов. Во входном и скрытом слоях мы использовали функцию активации relu, а на выходе - сигмоидальную функцию активации.
# model = Sequential()
# model.add(Dense(16, input_dim=len(features), activation='relu'))
# model.add(Dense(8, activation='relu'))
# model.add(Dense(1, activation='sigmoid'))

In [53]:

# compile the model
# model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

Разбиваем на выборки

In [29]:
X_train, X_test, y_train, y_test = train_test_split(tarin_df1[features], 
                                                    tarin_df1[target], 
                                                    test_size=0.2,
                                                   random_state=21, stratify=tarin_df1[target])

бейслайн модель

In [30]:
dummy_clf = DummyClassifier(strategy="most_frequent")
dummy_clf.fit(X_train, y_train)

DummyClassifier(strategy='most_frequent')

In [47]:
print('AUC for baseline set: %0.4f' % roc_auc_score(y_test, dummy_clf.predict_proba(X_test)[:, 1]))

AUC for baseline set: 0.5000


Ищкм наиболее эффектиный метод обучения, через рандом форест

In [32]:
rfc1 = RandomForestClassifier(max_depth=25, n_estimators=80, random_state=21)
rfc1.fit(X_train, y_train)

RandomForestClassifier(max_depth=25, n_estimators=80, random_state=21)

In [43]:
print('AUC for rfc set: %0.4f' %roc_auc_score(y_test, rfc1.predict_proba(X_test)[:, 1]))

AUC for rfc set: 0.8287


mlp модель

In [33]:
mlp = MLPClassifier(hidden_layer_sizes=(32,), random_state=21)
mlp.fit(X_train, y_train)

MLPClassifier(hidden_layer_sizes=(32,), random_state=21)

In [41]:
print('AUC for rfc set: %0.4f' % roc_auc_score(y_test, mlp.predict_proba(X_test)[:, 1]))

AUC for rfc set: 0.8209


In [155]:
roc_auc_score(y_train, rfc1.predict_proba(X_train)[:, 1])

0.9961492786318675

грид сетка для фореста

In [32]:
rfc = RandomForestClassifier(random_state=21)
param_grid = {'criterion': ['gini', 'entropy'],
              'max_depth': np.arange(21, 25),
              'n_estimators': np.arange(50, 80)}
gs = GridSearchCV(rfc, param_grid, scoring='accuracy', n_jobs=-1)
gs.fit(X_train, y_train)

GridSearchCV(estimator=RandomForestClassifier(random_state=21), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': array([21, 22, 23, 24]),
                         'n_estimators': array([50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66,
       67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79])},
             scoring='accuracy')

In [42]:
rfc = RandomForestClassifier(random_state=21)
param_grid = {'criterion': ['gini', 'entropy'],
              'max_depth': [22],
              'n_estimators': np.arange(50, 70)}
gs = GridSearchCV(rfc, param_grid, scoring='roc_auc', n_jobs=-1)
gs.fit(X_train, y_train)

GridSearchCV(estimator=RandomForestClassifier(random_state=21), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'], 'max_depth': [22],
                         'n_estimators': array([50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66,
       67, 68, 69])},
             scoring='roc_auc')

In [41]:
from sklearn.metrics import SCORERS
SCORERS.keys()

dict_keys(['explained_variance', 'r2', 'max_error', 'neg_median_absolute_error', 'neg_mean_absolute_error', 'neg_mean_absolute_percentage_error', 'neg_mean_squared_error', 'neg_mean_squared_log_error', 'neg_root_mean_squared_error', 'neg_mean_poisson_deviance', 'neg_mean_gamma_deviance', 'accuracy', 'top_k_accuracy', 'roc_auc', 'roc_auc_ovr', 'roc_auc_ovo', 'roc_auc_ovr_weighted', 'roc_auc_ovo_weighted', 'balanced_accuracy', 'average_precision', 'neg_log_loss', 'neg_brier_score', 'adjusted_rand_score', 'rand_score', 'homogeneity_score', 'completeness_score', 'v_measure_score', 'mutual_info_score', 'adjusted_mutual_info_score', 'normalized_mutual_info_score', 'fowlkes_mallows_score', 'precision', 'precision_macro', 'precision_micro', 'precision_samples', 'precision_weighted', 'recall', 'recall_macro', 'recall_micro', 'recall_samples', 'recall_weighted', 'f1', 'f1_macro', 'f1_micro', 'f1_samples', 'f1_weighted', 'jaccard', 'jaccard_macro', 'jaccard_micro', 'jaccard_samples', 'jaccard_wei

а вот и самая эффективная сборка

In [45]:
gs.best_params_

{'criterion': 'entropy', 'max_depth': 22, 'n_estimators': 69}

Оценка нам подходит

In [43]:
roc_auc_score(y_test, gs.best_estimator_.predict_proba(X_test)[:, 1])

0.8312937737551045

нейросеть на keras

In [34]:
# Мы использовали последовательную модель, которая представляет собой тип модели, в которой слои укладываются линейно. Первый слой - это слой ввода, где input_dim - это количество функций, а количество выходных единиц равно 16. В скрытом слое мы используем 8 единиц вывода. Наконец, на выходном уровне у нас есть только одна единица вывода, которая возвращает вероятность оттока клиентов. Во входном и скрытом слоях мы использовали функцию активации relu, а на выходе - сигмоидальную функцию активации.
model = Sequential()
model.add(Dense(16, input_dim=len(features), activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [35]:

# compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [36]:
model.fit(X_train, y_train, epochs=30, batch_size=100)

Epoch 1/30
2842/2842 [==============================] - 5s 1ms/step - loss: 0.2637 - accuracy: 0.9186
Epoch 2/30
2842/2842 [==============================] - 3s 1ms/step - loss: 0.2558 - accuracy: 0.9186
Epoch 3/30
2842/2842 [==============================] - 3s 1ms/step - loss: 0.2520 - accuracy: 0.9186
Epoch 4/30
2842/2842 [==============================] - 3s 1ms/step - loss: 0.2480 - accuracy: 0.9186
Epoch 5/30
2842/2842 [==============================] - 3s 962us/step - loss: 0.2443 - accuracy: 0.9186
Epoch 6/30
2842/2842 [==============================] - 2s 761us/step - loss: 0.2411 - accuracy: 0.9186
Epoch 7/30
2842/2842 [==============================] - 3s 1ms/step - loss: 0.2388 - accuracy: 0.9186
Epoch 8/30
2842/2842 [==============================] - 3s 996us/step - loss: 0.2368 - accuracy: 0.9186
Epoch 9/30
2842/2842 [==============================] - 3s 1ms/step - loss: 0.2355 - accuracy: 0.9186
Epoch 10/30
2842/2842 [==============================] - 3s 1ms/step - loss:

In [34]:
test_preds = [x[0] for x in model.predict(X_test)]

2220/2220 [==============================] - 1s 556us/step


In [35]:
print('AUC for keras set: %0.4f' % roc_auc_score(y_test, test_preds))

AUC for keras set: 0.7388


In [37]:
rfc = RandomForestClassifier(random_state=21)
param_grid = {'criterion': ['entropy'],
              'max_depth': [22],
              'n_estimators': [69]}
gs = GridSearchCV(rfc, param_grid, scoring='roc_auc', n_jobs=-1)
gs.fit(X_train, y_train)

GridSearchCV(estimator=RandomForestClassifier(random_state=21), n_jobs=-1,
             param_grid={'criterion': ['entropy'], 'max_depth': [22],
                         'n_estimators': [69]},
             scoring='roc_auc')

формируем таблицу результатов

In [38]:
class Table:
    def __init__(self, model: list):
        self.models = model
    def form_table(self):
        res = []
        for i in range(len(self.models)):
            if i == 3:
                auc = self.models[i][4]
            else:
                auc = self.add_auc(self.models[i][0])
            res.append({'librery': self.models[i][1], 'alorithm': self.models[i][2], 'parametrs': self.models[i][3], 'accuracy': self.add_accuracy(self.models[i][0], i), 'auc': auc})
        return(pd.DataFrame(res))
    def add_auc(self, model):
        try:
            test_preds = model.predict_proba(X_test)[:, 1]
        except:
            test_preds = [x[0] for x in model.predict(X_test)]
        return('%0.4f' % roc_auc_score(y_test, test_preds))
    def add_accuracy(self, model, i):
        print()
        # try:
            # test_preds = model.predict_proba(X_test)
        # except:
        print(i)
        if i == 4:
            print('ddddd')
            test_preds = [round(x[0]) for x in model.predict(X_test)]
        else:
            test_preds = model.predict(X_test)
        return('%0.4f' % accuracy_score(y_test, test_preds))

In [39]:
g = Table([(rfc1, 'sklearn.RandomForestClassifier', 'A random forest is a meta estimator that fits a number of decision tree classifiers on various sub-samples of the dataset and uses averaging to improve the predictive accuracy and control over-fitting. The sub-sample size is controlled with the max_samples parameter if bootstrap=True (default), otherwise the whole dataset is used to build each tree.', 'max_depth=25, n_estimators=80, random_state=21'), (mlp, 'sclrean.MLPClassifier', 'This model optimizes the log-loss function using LBFGS or stochastic gradient descent', 'hidden_layer_sizes=(32,), random_state=21'), (dummy_clf, 'skleran.DummyClassifier','DummyClassifier makes predictions that ignore the input features. This classifier serves as a simple baseline to compare against other more complex classifiers. The specific behavior of the baseline is selected with the strategy parameter', 'strategy="most_frequent"'), (gs, 'sklearn.GridSearchCV', 'Exhaustive search over specified parameter values for an estimator. Important members are fit, predict', "'criterion': ['gini', 'entropy'], 'max_depth': [22], 'n_estimators': np.arange(50, 70)", 0.8312), (model, 'keras.Sequential', 'The sequential API allows you to create models layer-by-layer for most problems. It is limited in that it does not allow you to create models that share layers or have multiple inputs or outputs', "loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']")])

In [40]:
fin_df = g.form_table()


0

1

2

3
2220/2220 [==============================] - 2s 587us/step

4
ddddd
2220/2220 [==============================] - 1s 579us/step


In [41]:
fin_df

,librery,alorithm,parametrs,accuracy,auc
0,sklearn.RandomForestClassifier,A random forest is a meta estimator that fits ...,"max_depth=25, n_estimators=80, random_state=21",0.9193,0.8295
1,sclrean.MLPClassifier,This model optimizes the log-loss function usi...,"hidden_layer_sizes=(32,), random_state=21",0.9167,0.8209
2,skleran.DummyClassifier,DummyClassifier makes predictions that ignore ...,"strategy=""most_frequent""",0.9186,0.5000
3,sklearn.GridSearchCV,Exhaustive search over specified parameter val...,"'criterion': ['gini', 'entropy'], 'max_depth':...",0.9194,0.8312
4,keras.Sequential,The sequential API allows you to create models...,"loss='binary_crossentropy', optimizer='adam', ...",0.9191,0.8141


формируем ксв файл для тестовой таблицы

In [ ]:
xx_train = test_df.drop(['TARGET'], axis=1)

In [ ]:
pred = gs.best_estimator_.predict(xx_train)
# csv_df1['predict'] = pred
xx_train['predict'] = pred
xx_train
xy_train = xx_train['predict']
xy_train

ID
400980    1
525062    0
280316    0
496066    0
375031    0
         ..
175305    0
275016    0
277398    0
469164    0
397854    1
Name: predict, Length: 88798, dtype: int64

In [81]:
xy_train.to_csv('data/new_test.csv', sep = ';')

In [55]:
# X_train = np.asarray(X_train).astype('float32')